## ECS 171 Team Project

# Background

# Dataset Attributes

In [70]:
!pip install nltk
!pip install textblob

In [71]:
import pandas as pd 
import numpy as np

# NLP Packages
from nltk.corpus import stopwords
from textblob import TextBlob 
from textblob import Word
import string
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

import re

In [72]:
dfTrain = pd.read_csv('data/train.csv')
dfTrain.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [73]:
dfTest = pd.read_csv('data/test.csv')
dfTest.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [74]:
punctuation='["\'?,\.]' # I will replace all these punctuation with ''

#Add any words you want to be replaced and the conversion
abbr_dict={
    "'til": "until",
    "'ll": " will",
    "y'all": "you all",
    "that's": "that is",
    "won't": "will not",
    "can't": "can not",
    "cannot": "can not",
    "ain't": "am not",
    "n't": " not", # wasn't weren't haven't didn't
    "'ve": " have", # i've we've you've would've
    "'d": " would", #you'd i'd we'd you'd
    "'re": " are", # they're we're you're
    "in'": "ing", # lookin'
    
    "what's":"what is",
    "what're":"what are",
    "who's":"who is",
    "who're":"who are",
    "where's":"where is",
    "where're":"where are",
    "when's":"when is",
    "when're":"when are",
    "how's":"how is",
    "how're":"how are",

    "i'm":"i am",
    "we're":"we are",
    "you're":"you are",
    "they're":"they are",
    "it's":"it is",
    "he's":"he is",
    "she's":"she is",
    "that's":"that is",
    "there's":"there is",
    "there're":"there are",

    "i've":"i have",
    "we've":"we have",
    "you've":"you have",
    "they've":"they have",
    "who've":"who have",
    "would've":"would have",
    "not've":"not have",

    "i'll":"i will",
    "we'll":"we will",
    "you'll":"you will",
    "he'll":"he will",
    "she'll":"she will",
    "it'll":"it will",
    "they'll":"they will",

    "isn't":"is not",
    "wasn't":"was not",
    "aren't":"are not",
    "weren't":"were not",
    "can't":"can not",
    "couldn't":"could not",
    "don't":"do not",
    "didn't":"did not",
    "shouldn't":"should not",
    "wouldn't":"would not",
    "doesn't":"does not",
    "haven't":"have not",
    "hasn't":"has not",
    "hadn't":"had not",
    "won't":"will not",
    "#": "",
    punctuation:'',
    '\s+':' ', # replace multi space with one single space
}

In [75]:
#Reads the data, converts to lowercase, and replaces using abbreviation list
def process_data(file_name):
    data=pd.read_csv(file_name)
    data.text=data.text.str.lower() # conver to lower case
    data.keyword=data.keyword.str.lower()
    data.text=data.text.astype(str)
    data.keyword=data.keyword.astype(str)
    data.replace(abbr_dict,regex=True,inplace=True)
    data.head(2)
    return data

In [76]:
train_df=process_data('data/train.csv')

In [77]:
train_df.head(10)

,id,keyword,location,text,target
0,1,nan,NaN,our deeds are the reason of this earthquake ma...,1
1,4,nan,NaN,forest fire near la ronge sask canada,1
2,5,nan,NaN,all residents asked to shelter in place are be...,1
3,6,nan,NaN,13000 people receive wildfires evacuation orde...,1
4,7,nan,NaN,just got sent this photo from ruby alaska as s...,1
5,8,nan,NaN,rockyfire update => california hwy 20 closed i...,1
6,10,nan,NaN,flood disaster heavy rain causes flash floodin...,1
7,13,nan,NaN,i am on top of the hill and i can see a fire i...,1
8,14,nan,NaN,there is an emergency evacuation happening now...,1
9,15,nan,NaN,i am afraid that the tornado is coming to our ...,1


In [78]:
sentence = train_df.iloc[19]
print(sentence)
print(sentence['text'])
words = ''.join(sentence['text']).split()
print(words)
[str(TextBlob(word).correct()) for word in words]

id                                    28
keyword                              nan
location                             NaN
text        what a goooooooaaaaaal!!!!!!
target                                 0
Name: 19, dtype: object
what a goooooooaaaaaal!!!!!!
['what', 'a', 'goooooooaaaaaal!!!!!!']


['what', 'a', 'goooooooaaaaaal!!!!!!']

In [79]:
#unique words
uniqueWords = list(set(" ".join(train_df['text']).lower().split(" ")))
count = len(uniqueWords)
print(count)

# Total words
train_df['total_words'] = train_df['text'].str.split().str.len()
print(train_df['total_words'].sum())

24563
114174


In [81]:
#remove http or https links from tweets

for sentence in range(len(train_df)):
    #print(train_df['text'][sentence])
    train_df['text'][sentence] = re.sub(r"http\S+", "", train_df['text'][sentence])

/var/folders/1k/mmxb6bc53wxbhjh1vhxj380c0000gn/T/ipykernel_28147/2956738419.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['text'][sentence] = re.sub(r"http\S+", "", train_df['text'][sentence])


In [84]:
#unique words
uniqueWords = list(set(" ".join(train_df['text']).lower().split(" ")))
count = len(uniqueWords)
print(count)

# Total words
train_df['total_words'] = train_df['text'].str.split().str.len()
print(train_df['total_words'].sum())


19973
109472
